In [30]:
#imports and reading in files + path setup
import os
import re
import pandas as pd
flatfiles=[]
for filename in os.listdir("genomes"):
    flatfiles.append(filename)
print(len(flatfiles))
path='C:\\Users\\Matt\\Desktop\\Bioinfo Project\\genomes\\'

467


In [31]:
def parseFile(filename, path):
    fileString=open(path+filename).read()
    #AQUIRE START INDICIES
    startIndex=fileString.find("CDS")
    starts=[]
    ends=[]
    while startIndex>0:
        temp=fileString[startIndex:startIndex+50]
        ex=r'[ (]\d+'
        match=re.search(ex,temp)
        try:
            starts.append(int(re.search(ex,temp).group(0)[1:]))
            ends.append(int(re.search(r'\.\.\d+',temp).group(0)[2:]))
        except:
            pass
        #print(filename)
        startIndex=fileString.find("CDS",startIndex+1)
    #print(starts)
    
    #ISOLATE GENOME
    genome=fileString[fileString.find("ORIGIN")+6:len(fileString)]
    genome=re.sub(r'[^a-z]+',"",genome)
    
    #CHANGE T BASE TO U TO REPRESENT AS RNA
    genomeRNA=genome.replace("t","u")
    
    #Get all start codons in genome
    startCodonIndicies=[]
    for i in range(ends[0],len(genomeRNA)):
        #print(genome[i:i+3])
        if genomeRNA[i:i+3]=='aug':
            startCodonIndicies.append(i+1)
    
    falsePositiveStarts=[]
    for index in startCodonIndicies:
        if index in starts:
            pass
        else:
            falsePositiveStarts.append(index)
    
    trueStarts=[]
    for i in range(len(starts)):
        if i==0: continue
        index=starts[i]
        code=genomeRNA[index-11:index+14]+filename
        if code[10:13]!="aug": 
            print(code)
            print(len(genome))
            print(len(genomeRNA))
            print(index)
            print(starts)
        trueStarts.append(genomeRNA[index-11:index+14]+filename+ " " + str(index))
    falseStarts=[]
    for index in falsePositiveStarts:
        falseStarts.append(genomeRNA[index-11:index+14]+filename+ " " + str(index))
    
    return {"trueStarts": trueStarts, "falseStarts": falseStarts}

In [32]:
print(parseFile(flatfiles[0],path))

{'trueStarts': ['cugaauugaaaugaaaugggguccaAF046869.txt 12073', 'auuuucagcaaugguuaauagcuguAF046869.txt 12696', 'uucaccuagaauggcuucgucccuuAF046869.txt 13241', 'auguuuaaguauguuggagaaaugcAF046869.txt 13788', 'cagcggaacaauggggucguccuuaAF046869.txt 14375', 'cuugucaaauaugccaaauaacaacAF046869.txt 14889'], 'falseStarts': ['agaccgaucgaugguggaguugugcAF046869.txt 7882', 'uccggugcugaugcaucucccaaguAF046869.txt 7948', 'cuugcccaucaugggccgggaaacaAF046869.txt 7975', 'acugggaucgauggcacgcucugggAF046869.txt 7999', 'aggcgcggcgaugcuccuaaaauugAF046869.txt 8092', 'ggccacgaccaugccccccggguuuAF046869.txt 8295', 'uuugaguuauauguaccgaccauacAF046869.txt 8317', 'ggcugcgaagaugccgcacugaaagAF046869.txt 8404', 'cucucuaaauaugacuuguccacccAF046869.txt 8431', 'cuguuugcccauguagguaagugccAF046869.txt 8500', 'uaagaauucuauggcugggauaaauAF046869.txt 8556', 'gcugggauaaaugggaacagguuccAF046869.txt 8569', 'ccaggguuucaugaaaaaggcguuuAF046869.txt 8781', 'aaccuucuuuaugaacuugccugugAF046869.txt 8935', 'uuggugaucuaugcacagcauauggAF046869.txt 90

In [20]:
#develop scoring matrix


#load true and false start spaces
trueStartSpace=[]
falseStartSpace=[]
#first 10 of flat files
for i in range(len(flatfiles)):
    if 'A'=='A':
        spaces=parseFile(flatfiles[i],path)
        trueStartSpace+=spaces["trueStarts"]
        falseStartSpace+=spaces["falseStarts"]

numNucs=25
#build consensus matrices
print("True start codon identities by position")
startIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in trueStartSpace:
    temp=i.replace(" ","").upper()
    for n in range(numNucs):
        try:
            startIdentities[temp[n]][n]+=1
        except:
            print("except: " + temp)
#make data frame of start identities for clean format of output
startIDsDF=pd.DataFrame.from_dict(startIdentities)
print(startIDsDF)

print("\nTrue start codon frequency by position")
for key in startIdentities:
    for i in range(numNucs):
        startIdentities[key][i]=round(startIdentities[key][i]/len(trueStartSpace),2)
startFreqsDF=pd.DataFrame.from_dict(startIdentities)
print(startFreqsDF.T)

print("\nFalse start codon identities by position")
falseStartIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in falseStartSpace:
    temp=i.replace(" ","").upper()
    try:
        for n in range(numNucs):
            falseStartIdentities[temp[n]][n]+=1
    except:
        pass
#make data frame of false start identities for clean format of output
falseStartIDsDF=pd.DataFrame.from_dict(falseStartIdentities)
print(falseStartIDsDF)

print("\nFalse start codon frequency by position")
for key in falseStartIdentities:
    for i in range(numNucs):
        falseStartIdentities[key][i]=round(falseStartIdentities[key][i]/len(falseStartSpace),2)
falseStartFreqsDF=pd.DataFrame.from_dict(falseStartIdentities)
print(falseStartFreqsDF.T)

True start codon identities by position
       A     C     G     U
0    929  1370    61   622
1    452   148    67  2315
2      7   575  1458   942
3   1026   509   478   969
4    580   491   525  1386
5    225  1354   539   864
6   1813    46    51  1072
7   1409     0  1551    22
8   1052   968   921    41
9   1836    12   134  1000
10  2982     0     0     0
11     0     0     0  2982
12     0     0  2982     0
13   467   465  1601   449
14   455  1317   675   535
15   976    79   983   944
16   905    35   930  1112
17   922  1048  1005     7
18    53    95  1777  1057
19  1412   335   522   713
20   947   942   888   205
21   237  1403   536   806
22   532  1042    79  1329
23   713   125   928  1216
24   901   976    71  1034

True start codon frequency by position
     0     1     2     3     4     5     6     7     8     9   ...     15  \
A  0.31  0.15  0.00  0.34  0.19  0.08  0.61  0.47  0.35  0.62  ...   0.33   
C  0.46  0.05  0.19  0.17  0.16  0.45  0.02  0.00  0.32  0.00  .

In [21]:
matrix=startFreqsDF
matrix.to_pickle("loadData\\allORFmatrix.pckl")
#score a sequence based on scoreing matrix
def scoreSequence(seq, matrix):
    positions=[0,1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21,22,23,24]
    weights=[3,3,1,0,1,1,4,4,2,3,0,0,0,1,1,2,2,2,4,2,1,1,2,2,2]
    seq=seq.upper()
    score=1
    for i in positions:
        try:
            positionScore=matrix[seq[i]][i]
        except:
            positionScore=0
        #for m in range(weights[i]):
        #    positionScore*=positionScore
        if positionScore==0:
            score*=.000001
        else:
            score*=positionScore
    return score

In [22]:
#true start scores
#print("True start Scores:")
trueHigh=0
trueLow=999
trueAVG=0
falseHigh=0
falseLow=999
falseAVG=0
#false start scores
print("False Start Scores:")
for i in range(len(falseStartSpace)):
    s=scoreSequence(falseStartSpace[i],matrix)
    #if s>.39: print(falseStartSpace[i] + str(s))
    if s>falseHigh: falseHigh=s
    if s<falseLow: falseLow=s
    falseAVG+=s
    #print(s)
falseAVG/=len(falseStartSpace)
print("True Start Scores: ")
for i in range(len(trueStartSpace)):
    s=scoreSequence(trueStartSpace[i],matrix)
    #if s<falseHigh: print(trueStartSpace[i] + str(s))
    if s>trueHigh: trueHigh=s
    if s<trueLow: trueLow=s
    trueAVG+=s
    #print(s)
trueAVG/=len(trueStartSpace)

print("True High: " + str(trueHigh))
print("True Low: "+ str(trueLow))
print("True AVG: " + str(trueAVG))
print("False High: " + str(falseHigh))
print("False Low: " + str(falseLow))
print("False AVG: "+ str(falseAVG))

print(trueLow>falseHigh)

with open("loadData\\allORFScoreThreshold.txt",'w') as f:
    f.write(str(trueLow))

False Start Scores:
True Start Scores: 
True High: 2.83286121436e-09
True Low: 2.9506113345e-22
True AVG: 3.77152198896e-10
False High: 6.98797069889e-11
False Low: 2.22110637581e-74
False AVG: 4.25358021437e-14
False


In [23]:
def parsePost(filename,path,lowScore):
    print(filename)
    fileString=open(path+filename).read()
    #AQUIRE START INDICIES
    startIndex=fileString.find("CDS")
    starts=[]
    ends=[]
    while startIndex>0:
        temp=fileString[startIndex:startIndex+50]
        ex=r'[ (]\d+'
        match=re.search(ex,temp)
        try:
            starts.append(int(re.search(ex,temp).group(0)[1:]))
            ends.append(int(re.search(r'\.\.\d+',temp).group(0)[2:]))
        except:
            pass
        #print(filename)
        startIndex=fileString.find("CDS",startIndex+1)
    #print(starts)
    
    #ISOLATE GENOME
    genome=""
    for char in fileString[fileString.find("ORIGIN"):len(fileString)]:
        if char=='t'or char=='g' or char=='a' or char=='c':
            genome+=char
    genome=fileString[fileString.find("ORIGIN")+6:len(fileString)]
    genome=re.sub(r'[^a-z]+',"",genome)
    
    #CHANGE T BASE TO U TO REPRESENT AS RNA
    genomeRNA=""
    for char in genome:
        if char=='t':
            genomeRNA+='u'
        else:
            genomeRNA+=char
    

    #find end of orf1b
    stopCodons=["uag","uga","uaa"]
    for i in range(ends[0]-1,len(genomeRNA),3):
        #print(genomeRNA[i:i+3])
        if genomeRNA[i:i+3] in stopCodons:
            end1b=i
            break
    ##testing
    def findNextStop(start,end):
        stopCodons=["uag","uga","uaa"]
        for i in range(start,end,3):
            if genomeRNA[i:i+3] in stopCodons:
                return i+3
    
    print(starts)
    #Get all start codons in genome
    startCodonIndicies=[]
    print(findNextStop(starts[0]-1,len(genomeRNA)))
    for i in range(end1b,len(genomeRNA)):
        #print(genome[i:i+3])
        if genomeRNA[i:i+3]=='aug':
            seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
            score=scoreSequence(seq,matrix)
            #print(seq)
            #print(str(i) + " " + str(score))
            if score>lowScore:
                #print(i+1)
                #print(starts[1])
                orf2index=i+1
                print(orf2index)
                print("ORF2: " + str(orf2index in starts))
                if (i+1!=starts[1]):
                    print(filename + " " + str(end1b))
                    print(str(i+1==starts[1]) + " " + str(i+1) + " " + genomeRNA[i+1:i+4])
                break
    #print(starts[1])
    #print(genomeRNA[starts[1]-11:starts[1]+14])
    for i in range(orf2index+1,len(genomeRNA)):
        if genomeRNA[i:i+3]=='aug':
            seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
            score=scoreSequence(seq,matrix)
            if score>lowScore:
                orf2bindex=i+1
                print(orf2bindex)
                print("ORF2b: " + str(orf2bindex in starts))
                break
    #LOCATE OPEN READING FRAME 3
    for i in range(findNextStop(orf2index-1,len(genomeRNA))-190,ends[1]):
        if genomeRNA[i:i+3]=='aug':
            seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
            score=scoreSequence(seq,matrix)
            if score>lowScore:
                orf3index=i+1
                print(orf3index)
                print("ORF3: " + str(orf3index in starts))
                break
    orf3end=findNextStop(orf3index-1,len(genomeRNA))
    #print(orf3end-250)
    orf4index=-1
    trace=0
    while orf4index==-1:
        for i in range(orf3end-(230+trace),orf3end-100):
            if genomeRNA[i:i+3]=='aug':
                seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
                score=scoreSequence(seq,matrix)
                #print(score)
                #print(i)
                if score>lowScore:
                    orf4index=i+1
                    print(orf4index)
                    print("ORF4: " + str(orf4index in starts))
                    break
        trace+=10
    #LOCATE OPEN READING FRAME 5
    orf4end=findNextStop(orf4index-1,len(genomeRNA))
    trace=0
    orf5index=-1
    while orf5index==-1:
        for i in range(orf4end+(3+trace),orf4end+60):
            if genomeRNA[i:i+3]=='aug':
                seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
                score=scoreSequence(seq,matrix)
                if score>lowScore:
                    orf5index=i+1
                    print(orf5index)
                    #print(str(i)+"i" + str(score))
                    print("ORF5: " + str(orf5index in starts))
                    break
        if trace==0: trace=-13
        elif trace==-13: trace=50
        else: break
    #LOCATE ORF 6
    for i in range(findNextStop(orf5index-1,len(genomeRNA))-30,len(genomeRNA)):
        if genomeRNA[i:i+3]=='aug':
            seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
            score=scoreSequence(seq,matrix)
            if score>lowScore:
                orf6index=i+1
                print(orf6index)
                print("ORF6: " + str(orf6index in starts))
                break
    #LOCATE ORF 7
    for i in range(findNextStop(orf6index-1,len(genomeRNA))-30,len(genomeRNA)):
        if genomeRNA[i:i+3]=='aug':
            seq=genomeRNA[i-10:i+15]+filename+ " " + str(i)
            score=scoreSequence(seq,matrix)
            if score>lowScore:
                orf7index=i+1
                print(orf7index)
                print("ORF7: " + str(orf7index in starts))
                break

In [24]:
for f in flatfiles:
    parsePost(f,path,trueLow)
#parsePost("AY262352.txt", path,trueLow)
#parsePost("KY767026.txt", path,trueLow)
#parsePost("KU131557.txt", path,trueLow)
#parsePost(flatfiles[67],path,trueLow)

AF046869.txt
[190, 12073, 12696, 13241, 13788, 14375, 14889]
7701
12073
ORF2: True
12078
ORF2b: False
12696
ORF3: True
13241
ORF4: True
13788
ORF5: True
14375
ORF6: True
14889
ORF7: True
AF066183.txt
[191, 12074, 12697, 13242, 13789, 14376, 14890]
7702
12074
ORF2: True
12079
ORF2b: False
12697
ORF3: True
13242
ORF4: True
13789
ORF5: True
14376
ORF6: True
14890
ORF7: True
AF159149.txt
[166, 12049, 12672, 13217, 13764, 14351, 14865]
7677
12049
ORF2: True
12054
ORF2b: False
12672
ORF3: True
13217
ORF4: True
13764
ORF5: True
14351
ORF6: True
14865
ORF7: True
AF184212.txt
[191, 12182, 12805, 13350, 13897, 14484, 14998]
7810
12182
ORF2: True
12187
ORF2b: False
12805
ORF3: True
13350
ORF4: True
13897
ORF5: True
14484
ORF6: True
14998
ORF7: True
AF494042.txt
[192, 12057, 12062, 12680, 13225, 13772, 14359, 14873]
7685
12057
ORF2: True
12062
ORF2b: True
12680
ORF3: True
13225
ORF4: True
13772
ORF5: True
14359
ORF6: True
14873
ORF7: True
AY150312.txt
[190, 12073, 12078, 12696, 13241, 13788]
7701


11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
EU880431.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
EU880432.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
EU880433.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
EU880434.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
EU880435.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: 

[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
GU168567.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
GU168568.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
GU168569.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
GU169411.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
GU232735.txt
[190, 11920, 12543, 13088, 13635, 14222, 14736]
7548
11920
ORF2: 

12605
ORF3: True
13150
ORF4: True
13697
ORF5: True
14284
ORF6: True
14798
ORF7: True
JN387273.txt
[189, 11880, 11885, 12503, 13048, 13595, 14182, 14696]
7508
11880
ORF2: True
11885
ORF2b: True
12503
ORF3: True
13048
ORF4: True
13595
ORF5: True
14182
ORF6: True
14696
ORF7: True
JN387274.txt
[189, 11880, 11885, 12503, 13048, 13595, 14182, 14696]
7508
11880
ORF2: True
11885
ORF2b: True
12503
ORF3: True
13048
ORF4: True
13595
ORF5: True
14182
ORF6: True
14696
ORF7: True
JN626287.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
JN654458.txt
[191, 12074, 12079, 12697, 13242, 13789, 14376, 14890]
7702
12074
ORF2: True
12079
ORF2b: True
12697
ORF3: True
13242
ORF4: True
13789
ORF5: True
14376
ORF6: True
14890
ORF7: True
JN654459.txt
[192, 11682, 11687, 12305, 12850, 13397, 13984, 14498]
7310
11682
ORF2: True
11687
ORF2b: True
12305
ORF3: True
12850
ORF4: True
13397
O

[189, 11982, 12605, 13150, 13697, 14284, 14798]
7610
11982
ORF2: True
11987
ORF2b: False
12605
ORF3: True
13150
ORF4: True
13697
ORF5: True
14284
ORF6: True
14798
ORF7: True
JX044140.txt
[191, 11639, 11644, 12262, 12807, 13354, 13941, 14455]
7267
11639
ORF2: True
11644
ORF2b: True
12262
ORF3: True
12807
ORF4: True
13354
ORF5: True
13941
ORF6: True
14455
ORF7: True
JX087437.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
JX177644.txt
[190, 11983, 11988, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: True
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
JX187609.txt
[221, 11780, 12388, 12930, 13475, 14068, 14579]
7396
11780
ORF2: True
11785
ORF2b: False
12388
ORF3: True
12930
ORF4: True
13475
ORF5: True
14068
ORF6: True
14579
ORF7: True
JX192632.txt
[190, 11983, 11988, 12606, 13151, 13698, 14285, 14799

[222, 11796, 11801, 12404, 12946, 13494, 14087, 14598]
7412
11796
ORF2: True
11920
ORF2b: False
12404
ORF3: True
12946
ORF4: True
13494
ORF5: True
14087
ORF6: True
14598
ORF7: True
KC862573.txt
[222, 11574, 11579, 12182, 12724, 13272, 13865, 14376]
7190
11574
ORF2: True
11579
ORF2b: True
12182
ORF3: True
12724
ORF4: True
13272
ORF5: True
13865
ORF6: True
14376
ORF7: True
KC862574.txt
[222, 11574, 11579, 12182, 12724, 13272, 13865, 14376]
7190
11574
ORF2: True
11579
ORF2b: True
12182
ORF3: True
12724
ORF4: True
13272
ORF5: True
13865
ORF6: True
14376
ORF7: True
KC862575.txt
[190, 12064, 12069, 12687, 13232, 13779, 14366, 14880]
7692
12064
ORF2: True
12069
ORF2b: True
12687
ORF3: True
13232
ORF4: True
13779
ORF5: True
14366
ORF6: True
14880
ORF7: True
KC862576.txt
[190, 12061, 12066, 12684, 13229, 13776, 14363, 14877]
7689
12061
ORF2: True
12066
ORF2b: True
12684
ORF3: True
13229
ORF4: True
13776
ORF5: True
14363
ORF6: True
14877
ORF7: True
KC862577.txt
[190, 12073, 12078, 12696, 13241, 

12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KJ534542.txt
[190, 11983, 11988, 12606, 13151, 13688, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: True
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KJ534543.txt
[190, 11983, 11988, 12606, 13151, 13688, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: True
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KJ546412.txt
[190, 11983, 11988, 12606, 13151, 13688, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: True
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KJ591659.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KJ609516.txt
[190, 11983, 11988, 12606, 13151, 13688, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: True
12606
ORF3: T

[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KP771767.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KP771768.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KP771769.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KP771770.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KP771771.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: 

12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KT033457.txt
[222, 11735, 11740, 12343, 12885, 13433, 14026, 14537]
7352
11735
ORF2: True
11740
ORF2b: True
12343
ORF3: True
12885
ORF4: True
13433
ORF5: True
14026
ORF6: True
14537
ORF7: True
KT159248.txt
[222, 11721, 11726, 12329, 12871, 13416, 13421, 14009, 14520]
7337
11721
ORF2: True
11726
ORF2b: True
12329
ORF3: True
12871
ORF4: True
13416
ORF5: True
14009
ORF6: True
14501
ORF7: False
KT159249.txt
[222, 11712, 11717, 12320, 12862, 13410, 13415, 14003, 14514]
7328
11712
ORF2: True
11717
ORF2b: True
12320
ORF3: True
12862
ORF4: True
13410
ORF5: True
14003
ORF6: True
14514
ORF7: True
KT180169.txt
[189, 11877, 12500, 13045, 13592, 14179, 14693]
7505
11877
ORF2: True
11882
ORF2b: False
12500
ORF3: True
13045
ORF4: True
13592
ORF5: True
14179
ORF6: True
14693
ORF7: True
KT224385.txt
[222, 11811, 11816, 12419, 12961, 13509, 14102, 14613]
7427
11811
ORF2: True
11816
ORF2b: True
12419
ORF3: True
12961
ORF

[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KX815417.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KX815418.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KX815419.txt
[191, 11681, 12304, 12849, 13396, 13986, 14500]
7309
11681
ORF2: True
11686
ORF2b: False
12304
ORF3: True
12849
ORF4: True
13396
ORF5: True
13986
ORF6: True
14500
ORF7: True
KX815420.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: True
11988
ORF2b: False
12606
ORF3: True
13151
ORF4: True
13698
ORF5: True
14285
ORF6: True
14799
ORF7: True
KX815421.txt
[190, 11983, 12606, 13151, 13698, 14285, 14799]
7611
11983
ORF2: 

In [25]:
tempPath=path='C:\\Users\\Matt\\Desktop\\Bioinfo Project\\testGenomes\\'

parsePost("MG011719.txt",tempPath,trueLow)

MG011719.txt
[192, 11682, 12305, 12850, 13397, 13984, 14498]
7310
11682
ORF2: True
11687
ORF2b: False
12305
ORF3: True
12850
ORF4: True
13397
ORF5: True
13984
ORF6: True
14498
ORF7: True
